In [1]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
from IPython.display import display, HTML
import sys
import math

In [59]:
yik_df = pd.read_csv('data/yik_df_1.csv').drop(columns=['Unnamed: 0'])
pd.options.display.max_colwidth = 200

In [60]:
yik_df.head()

,index,Число участников,Выданные бюллетени,Бюллетени в урнах,Недействительные бюллетени,Голосов за,Голосов против,ТИК,УИК,Явка%,за%,против%,Адрес,Полный адрес,Name,Location
0,УИК №1,2160,841,841,12,566,263,Александров,1,38.935185,67.300832,31.272295,"601657, Владимирская область, Александровский район, город Александров, улица Геологов, дом 1","601657, Владимирская область, Александровский район, город Александров, улица Геологов, дом 1, здание ГГЭ-30",здание ГГЭ-30,"('38.688679', '56.404742')"
1,УИК №2,812,278,278,0,158,120,Александров,2,34.236453,56.834532,43.165468,"601657, Владимирская область, Александровский район, город Александров, улица Двориковское шоссе, дом 1","601657, Владимирская область, Александровский район, город Александров, улица Двориковское шоссе, дом 1, здание средней общеобразовательной школы №5",здание средней общеобразовательной школы №5,"('38.709942', '56.405843')"
2,УИК №3,811,211,211,2,129,80,Александров,3,26.017263,61.137441,37.914692,"601650, Владимирская область, Александровский район, город Александров, улица Восстания 1905 года, дом 3","601650, Владимирская область, Александровский район, город Александров, улица Восстания 1905 года, дом 3, здание средней общеобразовательной школы №1",здание средней общеобразовательной школы №1,"('38.720335', '56.401843')"
3,УИК №4,1034,452,452,2,318,132,Александров,4,43.713733,70.353982,29.203540,"601653, Владимирская область, Александровский район, город Александров, улица Заводская, дом 13","601653, Владимирская область, Александровский район, город Александров, улица Заводская, дом 13, здание бывшего опорного пункта милиции",здание бывшего опорного пункта милиции,"('38.728851', '56.413398')"
4,УИК №5,1948,854,854,2,662,190,Александров,5,43.839836,77.517564,22.248244,"601650, Владимирская область, Александровский район, город Александров, улица Стрелецкая набережная, дом 1","601650, Владимирская область, Александровский район, город Александров, улица Стрелецкая набережная, дом 1, здание МБУК ""Клуб ""Искож""","здание МБУК ""Клуб ""Искож""","('38.729624', '56.392433')"


##### Загрузка адресов УИКов

##### Создание словаря по локации(Тест)

In [51]:
loc_yik

{('40.380054', '56.13429'): [345, 384],
 ('40.347139', '56.114335'): [346, 354, 355, 392],
 ('40.36409', '56.137796'): [347],
 ('40.36215', '56.130442'): [348, 349],
 ('40.359114', '56.129088'): [350],
 ('40.353472', '56.123197'): [351, 352, 386],
 ('40.352844', '56.117215'): [353],
 ('40.340159', '56.112006'): [356, 357, 358, 393],
 ('40.349017', '56.110314'): [359, 388],
 ('40.341687', '56.107127'): [360, 381, 395],
 ('40.350274', '56.107262'): [361, 362, 363],
 ('40.361009', '56.116116'): [364, 394],
 ('40.359716', '56.115725'): [365, 366],
 ('40.373846', '56.11182'): [367, 385],
 ('40.378976', '56.120216'): [368],
 ('40.371079', '56.118134'): [369],
 ('40.380458', '56.122269'): [370, 397],
 ('40.38698', '56.126378'): [371],
 ('40.380566', '56.123568'): [372],
 ('40.395783', '56.126845'): [373, 399],
 ('40.391687', '56.130512'): [374],
 ('40.281365', '56.108568'): [375, 377, 382],
 ('40.283018', '56.104968'): [376, 389],
 ('40.28114', '56.101349'): [378],
 ('40.198279', '56.096669')

##### Определение координат УИКов по адресу

##### Наименование учреждения, столбец "Name"

##### Создание словаря по локации

In [53]:
loc_yik = {}

for i in range(yik_df.shape[0]):
    if yik_df['Location'][i] != 'Not defined':
        point = tuple(yik_df['Location'][i])
        if point in loc_yik:
            loc_yik[point].append(yik_df['УИК'][i])
        else:
            loc_yik[point] = [yik_df['УИК'][i]]

In [54]:
loc_yik

{('38.688679', '56.404742'): [1],
 ('38.709942', '56.405843'): [2],
 ('38.720335', '56.401843'): [3, 19],
 ('38.728851', '56.413398'): [4],
 ('38.729624', '56.392433'): [5],
 ('38.72851', '56.40006'): [6],
 ('38.719554', '56.387475'): [7],
 ('38.745389', '56.401201'): [8, 25],
 ('38.719392', '56.375204'): [9],
 ('38.719203', '56.377223'): [10],
 ('38.709735', '56.37515'): [11, 13],
 ('38.715107', '56.381704'): [12],
 ('38.710274', '56.384166'): [14, 15],
 ('38.715933', '56.393065'): [16],
 ('38.718278', '56.397235'): [17, 18],
 ('38.723955', '56.399532'): [20],
 ('38.704543', '56.394286'): [21, 64],
 ('38.711029', '56.397544'): [22],
 ('38.680064', '56.384076'): [23],
 ('38.688436', '56.388461'): [24],
 ('38.847833', '56.507149'): [26],
 ('38.833191', '56.50167'): [27],
 ('38.841042', '56.506146'): [28],
 ('38.837988', '56.463854'): [29],
 ('38.706717', '56.44927'): [30],
 ('38.600689', '56.416749'): [31],
 ('38.479111', '56.384555'): [32],
 ('38.531446', '56.544396'): [33],
 ('38.5400

##### Сохранение датафрейма УИКов

In [56]:
yik_df.drop(columns=['Unnamed: 0']).to_csv('data/yik_df_1.csv')